# Generating C++ Kokkos Chemistry Solvers with JAFF

This notebook demonstrates how to use JAFF to generate C++ code that uses the Kokkos-kernels BDF solver for solving chemical reaction networks. We'll use the `react_COthin` network as an example, which contains 37 species.

## Overview

The Kokkos output plugin generates:
- **chemistry_ode.hpp**: Chemistry ODE system class with evaluate_function and evaluate_jacobian methods
- **chemistry_ode.cpp**: Main executable that sets up and runs the BDF solver
- **CMakeLists.txt**: Build configuration for Kokkos and Kokkos-kernels dependencies

The generated code is compatible with the Kokkos-kernels BDF solver for stiff ODEs.

## Step 1: Load the Chemical Network

First, let's load the react_COthin network, which is a carbon-oxygen chemistry network in KROME format.

In [1]:
import os

from jaff.builder import Builder
from jaff.network import Network

# Load the test2 network
network = Network("../networks/react_COthin")

print(f"Network loaded: {network.label}")
print(f"Number of species: {network.get_number_of_species()}")
print(f"Number of reactions: {len(network.reactions)}")

Welcome to JAFF: Just Another Fancy Format!
Loading network from ../networks/react_COthin
Network label = react_COthin


  3%|███▌                                                                                                                                       | 9/350 [00:00<00:03, 86.11it/s]

KROME variable detected: @var:Hnuclei = get_Hnuclei(n(:))
KROME variable detected: @var: Te = Tgas*8.617343d-5
KROME variable detected: @var: invT = 1d0/Tgas
KROME variable detected: @var: lnTe = log(Te)
KROME variable detected: @var: T = Tgas
KROME format detected: @format:idx,R,R,R,P,P,P,P,Tmin,Tmax,rate


  5%|███████                                                                                                                                   | 18/350 [00:00<00:07, 45.75it/s]

KROME format detected: @format:idx,R,R,R,P,P,P,Tmin,Tmax,rate
KROME variable detected: @var:invTe=1d0/Te
KROME variable detected: @var:logT=log10(Tgas)
KROME variable detected: @var:invsqrT=1d0/sqrt(Tgas)
KROME variable detected: @var:kl11 = 1d1**(-27.029d0+3.801d0*logT-29487d0*invT)
KROME variable detected: @var:kh11 = 1d1**(-2.729d0-1.75d0*logT-23474d0*invT)
KROME variable detected: @var:ncr11 = 1d1**(5.0792d0*(1d0-1.23d-5*(Tgas-2d3)))
KROME variable detected: @var:a11=1.d0/(1.d0+(Hnuclei/(ncr11+1d-40)))


...

KROME format detected: @format:idx,R,R,P,P,P,Tmin,Tmax,rate
KROME format detected: @format:idx,R,R,P,P,Tmin,Tmax,rate
KROME format detected: @format:idx,R,R,P,P,P,Tmin,Tmax,rate
KROME format detected: @format:idx,R,R,P,P,Tmin,Tmax,rate
KROME format detected: @format:idx,R,R,P,P,P,Tmin,Tmax,rate
KROME format detected: @format:idx,R,R,P,P,Tmin,Tmax,rate
KROME format detected: @format:idx,R,R,P,P,P,Tmin,Tmax,rate
KROME format detected: @format:idx,R,R,P,P,Tmin,Tmax,rate
KROME format detected: @format:idx,R,R,P,P,P,Tmin,Tmax,rate
KROME format detected: @format:idx,R,R,P,P,Tmin,Tmax,rate
KROME format detected: @format:idx,R,R,P,P,P,Tmin,Tmax,rate
KROME format detected: @format:idx,R,R,P,P,Tmin,Tmax,rate
KROME format detected: @format:idx,R,R,P,P,P,Tmin,Tmax,rate
KROME format detected: @format:idx,R,R,P,P,Tmin,Tmax,rate
KROME format detected: @format:idx,R,R,P,P,P,Tmin,Tmax,rate
KROME format detected: @format:idx,R,R,P,P,Tmin,Tmax,rate
KROME format detected: @format:idx,R,R,P,P,P,Tmin,Tmax,r

 86%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████▉                   | 301/350 [00:02<00:00, 213.07it/s]

KROME format detected: @format:idx,R,R,P,rate
KROME variable detected: @var:fA = 1d0/(1d0+1d4*exp(-6d2/(user_Tdust+1d-40)))
KROME format detected: @format:idx,R,P,P,rate
KROME variable detected: @var: user_H2self = fselfH2(1.87d21*(merge(n(idx_H2),n_global(idx_H2),n(idx_H2) > 0.0_8)*1d-3)**(2./3.), 1d5)
KROME variable detected: @var: HnOj = fHnOj(user_Av)


100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 350/350 [00:02<00:00, 121.52it/s]


KROME format detected: @format:idx,R,P,P,P,rate
KROME variable detected: @var:ntot=sum(n(1:nmols))
KROME format detected: @format:idx,R,R,P,Tmin,Tmax,rate
Variables found: ['av', 'crate', 'tdust', 'tgas']
Loaded 287 reactions
Lodaded 0 photo-chemistry reactions
All done!
Network loaded: react_COthin
Number of species: 37
Number of reactions: 287


## Step 2: Examine Network Properties

Let's examine some properties of the network to understand what we're working with.

In [2]:
for i, species in enumerate(network.species):
    print(f"  {i}: {species.name} (mass: {species.mass:.2e}, charge: {species.charge})")

print(f"\nTotal species: {len(network.species)}")

  0: H (mass: 1.67e-24, charge: 0)
  1: e- (mass: 9.11e-28, charge: -1)
  2: H+ (mass: 1.67e-24, charge: 1)
  3: He (mass: 6.65e-24, charge: 0)
  4: He+ (mass: 6.65e-24, charge: 1)
  5: He++ (mass: 6.65e-24, charge: 2)
  6: H2 (mass: 3.35e-24, charge: 0)
  7: H2+ (mass: 3.35e-24, charge: 1)
  8: H- (mass: 1.67e-24, charge: -1)
  9: C+ (mass: 1.99e-23, charge: 1)
  10: C (mass: 1.99e-23, charge: 0)
  11: Si+ (mass: 4.66e-23, charge: 1)
  12: Si (mass: 4.66e-23, charge: 0)
  13: O+ (mass: 2.66e-23, charge: 1)
  14: O (mass: 2.66e-23, charge: 0)
  15: Si++ (mass: 4.66e-23, charge: 2)
  16: OH (mass: 2.82e-23, charge: 0)
  17: HOC+ (mass: 4.82e-23, charge: 1)
  18: HCO+ (mass: 4.82e-23, charge: 1)
  19: CO (mass: 4.65e-23, charge: 0)
  20: CH (mass: 2.16e-23, charge: 0)
  21: CH2 (mass: 2.33e-23, charge: 0)
  22: C2 (mass: 3.99e-23, charge: 0)
  23: HCO (mass: 4.82e-23, charge: 0)
  24: H2O (mass: 2.99e-23, charge: 0)
  25: O2 (mass: 5.31e-23, charge: 0)
  26: H3+ (mass: 5.02e-24, charge: 

In [3]:
# Show a few example reactions
for i, reaction in enumerate(network.reactions[:10]):
    reactants = " + ".join([f"{r.name}" for r in reaction.reactants])
    products = " + ".join([f"{p.name}" for p in reaction.products])
    print(f"  {i + 1}: {reactants} -> {products}")
    print(f"      Rate: {reaction.rate}")
    print()

  1: H + e- -> H+ + e- + e-
      Rate: 5.90824386372651e-70*exp(-2.03914985e-6*log(8.617343e-5*Max(2.73, Min(100000000.0, tgas)))**8 + 0.000111954395*log(8.617343e-5*Max(2.73, Min(100000000.0, tgas)))**7 - 0.00263197617*log(8.617343e-5*Max(2.73, Min(100000000.0, tgas)))**6 + 0.0348255977*log(8.617343e-5*Max(2.73, Min(100000000.0, tgas)))**5 - 0.2877056*log(8.617343e-5*Max(2.73, Min(100000000.0, tgas)))**4 + 1.56315498*log(8.617343e-5*Max(2.73, Min(100000000.0, tgas)))**3 - 5.73932875*log(8.617343e-5*Max(2.73, Min(100000000.0, tgas)))**2)*Max(2.73, Min(100000000.0, tgas))**13.536556

  2: H+ + e- -> H
      Rate: 1.49810881307214e-10/Max(2.73, Min(5500.0, tgas))**0.6353

  3: H+ + e- -> H
      Rate: 3.286733702438273e-10*exp(-3.071135243196595e-9*log(8.617343e-5*Max(5500.0, Min(100000000.0, tgas)))**9 - 1.856767039775261e-8*log(8.617343e-5*Max(5500.0, Min(100000000.0, tgas)))**8 + 5.755614137575758e-7*log(8.617343e-5*Max(5500.0, Min(100000000.0, tgas)))**7 + 4.989108920299513e-6*log(8

## Step 3: Generate C++ Kokkos Code

Now let's use the Builder to generate C++ code using the `kokkos_ode` template.

In [4]:
# Create builder and generate Kokkos C++ code
builder = Builder(network)
builds_dir = builder.build(template="kokkos_ode")

print("\nC++ Kokkos code generated successfully!")
print(f"Generated files are located at {builds_dir}")

Building network with template: kokkos_ode
Preprocessing /home/anish/Programming/research/jaff/codegen_class/src/jaff/templates/kokkos_ode/chemistry_ode.hpp -> /home/anish/Programming/research/jaff/codegen_class/examples/chemistry_ode.hpp
Preprocessing /home/anish/Programming/research/jaff/codegen_class/src/jaff/templates/kokkos_ode/chemistry_ode.cpp -> /home/anish/Programming/research/jaff/codegen_class/examples/chemistry_ode.cpp
Preprocessing /home/anish/Programming/research/jaff/codegen_class/src/jaff/templates/kokkos_ode/CMakeLists.txt -> /home/anish/Programming/research/jaff/codegen_class/examples/CMakeLists.txt
Network built successfully using template 'kokkos_ode'.
Output files are located in: /home/anish/Programming/research/jaff/codegen_class/examples

C++ Kokkos code generated successfully!
Generated files are located at /home/anish/Programming/research/jaff/codegen_class/examples


## Step 4: Examine Generated Files

Let's look at the structure and content of the generated files.

In [5]:
# List generated files
generated_files = [
    f for f in os.listdir(builds_dir) if f.endswith((".cpp", ".hpp", ".txt"))
]

print("Generated files:")
for file in sorted(generated_files):
    filepath = os.path.join(builds_dir, file)
    size = os.path.getsize(filepath)
    print(f"  {file:20} ({size:,} bytes)")

Generated files:
  CMakeLists.txt       (2,371 bytes)
  chemistry_ode.cpp    (9,915 bytes)
  chemistry_ode.hpp    (204,786 bytes)


## Step 5: Understanding the Generated Code Structure

Let's examine key components of the generated chemistry solver.

In [6]:
# Show the ChemistryODE class definition
with open(builds_dir + "/chemistry_ode.hpp", "r") as f:
    lines = f.readlines()

# Find the class definition
class_start = None
for i, line in enumerate(lines):
    if "struct ChemistryODE" in line:
        class_start = i
        break

if class_start:
    print("ChemistryODE class structure:")
    print("=" * 50)
    # Show class definition and methods
    brace_count = 0
    for i in range(class_start, len(lines)):
        line = lines[i].rstrip()
        print(f"{i + 1:2d}: {line}")

        if "{" in line:
            brace_count += line.count("{")
        if "}" in line:
            brace_count -= line.count("}")
            if brace_count == 0:
                break

ChemistryODE class structure:
14: struct ChemistryODE {
15:     // Number of species in the chemical network
16:     // PREPROCESS_NUM_SPECIES
17: 
18:     static constexpr int neqs = 37;
19: 
20:     // PREPROCESS_END
21: 
22:     using state_type = std::array<integrators::Real, neqs>;
23:     using rhs_type = std::array<integrators::Real, neqs>;
24:     using jacobian_type = std::array<std::array<integrators::Real, neqs>, neqs>;
25: 
26:     // Species indices and common constants
27:     // PREPROCESS_COMMONS
28: 
29:     static constexpr int idx_h = 0;
30:     static constexpr int idx_e = 1;
31:     static constexpr int idx_hj = 2;
32:     static constexpr int idx_he = 3;
33:     static constexpr int idx_hej = 4;
34:     static constexpr int idx_hejj = 5;
35:     static constexpr int idx_h2 = 6;
36:     static constexpr int idx_h2j = 7;
37:     static constexpr int idx_hk = 8;
38:     static constexpr int idx_cj = 9;
39:     static constexpr int idx_c = 10;
40:     static constexpr

## Step 6: Building and Running Instructions

Here's how to build and run the generated C++ code:

* **Configure the build with CMake:**

In [16]:
!   cmake -S . -B build -DCMAKE_CXX_COMPILER=clang++

-- The Fortran compiler identification is GNU 15.2.1
-- Detecting Fortran compiler ABI info
-- Detecting Fortran compiler ABI info - done
-- Check for working Fortran compiler: /usr/bin/gfortran - skipped
-- Looking for a CUDA compiler
-- Looking for a CUDA compiler - NOTFOUND
-- Configuring done (1.1s)
-- Generating done (0.0s)
-- Build files have been written to: /home/anish/Programming/research/jaff/codegen_class/examples/build


In [17]:
assert _exit_code == 0

* **Build with parallel compilation:**

In [18]:
!   cmake --build build -j

[ 50%] Building CXX object CMakeFiles/chemistry_ode.dir/chemistry_ode.cpp.o
[100%] Linking CXX executable chemistry_ode
[100%] Built target chemistry_ode


In [19]:
assert _exit_code == 0

* **Run the chemistry solver:**

In [20]:
!   ./build/chemistry_ode --analytic-jac

Solving Chemistry ODE System (header-only integrators)
Number of species: 37
Number of reactions: 287
Time interval: [0, 3.15576e+09] seconds
Initial conditions:
  nden[0] = 1e-06
  nden[1] = 1e-10
  nden[2] = 1e-10
  nden[3] = 1e-10
  nden[4] = 1e-10
  nden[5] = 1e-10
  nden[6] = 1e-10
  nden[7] = 1e-10
  nden[8] = 1e-10
  nden[9] = 1e-10
  nden[10] = 1e-10
  nden[11] = 1e-10
  nden[12] = 1e-10
  nden[13] = 1e-10
  nden[14] = 1e-10
  nden[15] = 1e-10
  nden[16] = 1e-10
  nden[17] = 1e-10
  nden[18] = 1e-10
  nden[19] = 1e-10
  nden[20] = 1e-10
  nden[21] = 1e-10
  nden[22] = 1e-10
  nden[23] = 1e-10
  nden[24] = 1e-10
  nden[25] = 1e-10
  nden[26] = 1e-10
  nden[27] = 1e-10
  nden[28] = 1e-10
  nden[29] = 1e-10
  nden[30] = 1e-10
  nden[31] = 1e-10
  nden[32] = 1e-10
  nden[33] = 1e-10
  nden[34] = 1e-10
  nden[35] = 1e-10
  nden[36] = 1e-10
Jacobian: analytic

Final solution at t = 3155760000 seconds:
  nden[0] = 1.000355681e-06
  nden[1] = 4.37593792e-10
  nden[2] = 1.520577576e-10


In [21]:
assert _exit_code == 0

* **Or, run with custom time interval (1e12 seconds):**

In [22]:
!   ./build/chemistry_ode --analytic-jac 1e12

Solving Chemistry ODE System (header-only integrators)
Number of species: 37
Number of reactions: 287
Time interval: [0, 1e+12] seconds
Initial conditions:
  nden[0] = 1e-06
  nden[1] = 1e-10
  nden[2] = 1e-10
  nden[3] = 1e-10
  nden[4] = 1e-10
  nden[5] = 1e-10
  nden[6] = 1e-10
  nden[7] = 1e-10
  nden[8] = 1e-10
  nden[9] = 1e-10
  nden[10] = 1e-10
  nden[11] = 1e-10
  nden[12] = 1e-10
  nden[13] = 1e-10
  nden[14] = 1e-10
  nden[15] = 1e-10
  nden[16] = 1e-10
  nden[17] = 1e-10
  nden[18] = 1e-10
  nden[19] = 1e-10
  nden[20] = 1e-10
  nden[21] = 1e-10
  nden[22] = 1e-10
  nden[23] = 1e-10
  nden[24] = 1e-10
  nden[25] = 1e-10
  nden[26] = 1e-10
  nden[27] = 1e-10
  nden[28] = 1e-10
  nden[29] = 1e-10
  nden[30] = 1e-10
  nden[31] = 1e-10
  nden[32] = 1e-10
  nden[33] = 1e-10
  nden[34] = 1e-10
  nden[35] = 1e-10
  nden[36] = 1e-10
Jacobian: analytic

Final solution at t = 1e+12 seconds:
  nden[0] = 1.00145815e-06
  nden[1] = 1.466593952e-09
  nden[2] = 6.996983019e-10
  nden[3] =

In [23]:
assert _exit_code == 0